In [47]:
import numpy as np

In [48]:
#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.90
TOTAL_IMAGE_COUNT = 8
IMAGE_COUNT = int(TOTAL_IMAGE_COUNT/4)
NUM_VARIETIES = 4
NUM_OF_BANDS = 3
FIRST_BAND = 21
LAST_BAND = 149

IMAGE_WIDTH = 50
IMAGE_HEIGHT = 50

ALPHA = 0.1
BATCH_SIZE = 2*NUM_VARIETIES
LEARNING_RATE_BASE = 0.0001

In [49]:
def dataset_file_name():
    return "./dataset/V"+str(NUM_VARIETIES).zfill(3)+"_IC_"+str(TOTAL_IMAGE_COUNT).zfill(5)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)

In [50]:
DATASET_FILE_NAME = dataset_file_name()
train_dataset = np.load(DATASET_FILE_NAME+"_train_dataset.npy")
train_dataset_label = np.load(DATASET_FILE_NAME+"_train_dataset_label.npy")
test_dataset = np.load(DATASET_FILE_NAME+"_test_dataset.npy")
test_dataset_label = np.load(DATASET_FILE_NAME+"_test_dataset_label.npy")

In [51]:
"""MobileNet v2 models for Keras.
# Reference
- [Inverted Residuals and Linear Bottlenecks Mobile Networks for
   Classification, Detection and Segmentation]
   (https://arxiv.org/abs/1801.04381)
"""

'MobileNet v2 models for Keras.\n# Reference\n- [Inverted Residuals and Linear Bottlenecks Mobile Networks for\n   Classification, Detection and Segmentation]\n   (https://arxiv.org/abs/1801.04381)\n'

In [52]:
from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
from keras.layers import Activation, BatchNormalization, Add, Reshape, DepthwiseConv2D
from keras.utils.vis_utils import plot_model
from keras import backend as K

In [53]:
def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

In [54]:
def relu6(x):
    """Relu 6
    """
    return K.relu(x, max_value=6.0)

In [55]:
def _conv_block(inputs, filters, kernel, strides):
    """Convolution Block
    This function defines a 2D convolution operation with BN and relu6.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)
    return Activation(relu6)(x)

In [56]:
def _bottleneck(inputs, filters, kernel, t, alpha, s, r=False):
    """Bottleneck
    This function defines a basic bottleneck structure.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        alpha: Integer, width multiplier.
        r: Boolean, Whether to use the residuals.
    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    # Depth
    tchannel = K.int_shape(inputs)[channel_axis] * t
    # Width
    cchannel = int(filters * alpha)

    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))

    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation(relu6)(x)

    x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)

    if r:
        x = Add()([x, inputs])

    return x

In [57]:
def _inverted_residual_block(inputs, filters, kernel, t, alpha, strides, n):
    """Inverted Residual Block
    This function defines a sequence of 1 or more identical layers.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        alpha: Integer, width multiplier.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        n: Integer, layer repeat times.
    # Returns
        Output tensor.
    """

    x = _bottleneck(inputs, filters, kernel, t, alpha, strides)

    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, alpha, 1, True)

    return x

In [58]:
def MobileNetv2(input_shape, k, alpha=1.0):
    """MobileNetv2
    This function defines a MobileNetv2 architectures.
    # Arguments
        input_shape: An integer or tuple/list of 3 integers, shape
            of input tensor.
        k: Integer, number of classes.
        alpha: Integer, width multiplier, better in [0.35, 0.50, 0.75, 1.0, 1.3, 1.4].
    # Returns
        MobileNetv2 model.
    """
    inputs = Input(shape=input_shape)

    first_filters = _make_divisible(32 * alpha, 8)
    x = _conv_block(inputs, first_filters, (3, 3), strides=(2, 2))

    x = _inverted_residual_block(x, 16, (3, 3), t=1, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 24, (3, 3), t=6, alpha=alpha, strides=2, n=2)
    x = _inverted_residual_block(x, 32, (3, 3), t=6, alpha=alpha, strides=2, n=3)
    x = _inverted_residual_block(x, 64, (3, 3), t=6, alpha=alpha, strides=2, n=4)
    x = _inverted_residual_block(x, 96, (3, 3), t=6, alpha=alpha, strides=1, n=3)
    x = _inverted_residual_block(x, 160, (3, 3), t=6, alpha=alpha, strides=2, n=3)
    x = _inverted_residual_block(x, 320, (3, 3), t=6, alpha=alpha, strides=1, n=1)

    if alpha > 1.0:
        last_filters = _make_divisible(1280 * alpha, 8)
    else:
        last_filters = 1280

    x = _conv_block(x, last_filters, (1, 1), strides=(1, 1))
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, last_filters))(x)
    x = Dropout(0.3, name='Dropout')(x)
    x = Conv2D(k, (1, 1), padding='same')(x)

    x = Activation('softmax', name='softmax')(x)
    output = Reshape((k,))(x)

    model = Model(inputs, output)

    return model

In [59]:
model = MobileNetv2((IMAGE_HEIGHT, IMAGE_WIDTH, NUM_OF_BANDS), NUM_VARIETIES, ALPHA)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 50, 50, 3)]  0           []                               
                                                                                                  
 conv2d_37 (Conv2D)             (None, 25, 25, 8)    224         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_53 (BatchN  (None, 25, 25, 8)   32          ['conv2d_37[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_36 (Activation)     (None, 25, 25, 8)    0           ['batch_normalization_53[0]

 Conv2D)                                                                                          
                                                                                                  
 batch_normalization_64 (BatchN  (None, 7, 7, 12)    48          ['depthwise_conv2d_20[0][0]']    
 ormalization)                                                                                    
                                                                                                  
 activation_44 (Activation)     (None, 7, 7, 12)     0           ['batch_normalization_64[0][0]'] 
                                                                                                  
 conv2d_45 (Conv2D)             (None, 7, 7, 3)      39          ['activation_44[0][0]']          
                                                                                                  
 batch_normalization_65 (BatchN  (None, 7, 7, 3)     12          ['conv2d_45[0][0]']              
 ormalizat

 activation_51 (Activation)     (None, 4, 4, 36)     0           ['batch_normalization_75[0][0]'] 
                                                                                                  
 depthwise_conv2d_24 (Depthwise  (None, 4, 4, 36)    360         ['activation_51[0][0]']          
 Conv2D)                                                                                          
                                                                                                  
 batch_normalization_76 (BatchN  (None, 4, 4, 36)    144         ['depthwise_conv2d_24[0][0]']    
 ormalization)                                                                                    
                                                                                                  
 activation_52 (Activation)     (None, 4, 4, 36)     0           ['batch_normalization_76[0][0]'] 
                                                                                                  
 conv2d_53

                                                                                                  
 conv2d_60 (Conv2D)             (None, 4, 4, 54)     540         ['batch_normalization_86[0][0]'] 
                                                                                                  
 batch_normalization_87 (BatchN  (None, 4, 4, 54)    216         ['conv2d_60[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_59 (Activation)     (None, 4, 4, 54)     0           ['batch_normalization_87[0][0]'] 
                                                                                                  
 depthwise_conv2d_28 (Depthwise  (None, 4, 4, 54)    540         ['activation_59[0][0]']          
 Conv2D)                                                                                          
          

                                                                                                  
 batch_normalization_98 (BatchN  (None, 2, 2, 16)    64          ['conv2d_67[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_18 (Add)                   (None, 2, 2, 16)     0           ['batch_normalization_98[0][0]', 
                                                                  'batch_normalization_95[0][0]'] 
                                                                                                  
 conv2d_68 (Conv2D)             (None, 2, 2, 96)     1632        ['add_18[0][0]']                 
                                                                                                  
 batch_normalization_99 (BatchN  (None, 2, 2, 96)    384         ['conv2d_68[0][0]']              
 ormalizat

In [60]:
import timeit
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [61]:
from keras.optimizers import Adam
adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam_opt, metrics=['accuracy'])

In [62]:
tic = start_timer()
model.fit(x=train_dataset,y=train_dataset_label,batch_size=BATCH_SIZE, epochs=40, initial_epoch = 0, verbose=2,shuffle=True)
toc = end_timer()
show_time(tic,toc)

Testing started
Epoch 1/40
2/2 - 17s - loss: 1.6772 - accuracy: 0.1250 - 17s/epoch - 9s/step
Epoch 2/40
2/2 - 0s - loss: 1.5837 - accuracy: 0.1875 - 121ms/epoch - 61ms/step
Epoch 3/40
2/2 - 0s - loss: 1.3606 - accuracy: 0.2500 - 121ms/epoch - 60ms/step
Epoch 4/40
2/2 - 0s - loss: 1.2202 - accuracy: 0.5625 - 135ms/epoch - 68ms/step
Epoch 5/40
2/2 - 0s - loss: 1.3711 - accuracy: 0.3750 - 125ms/epoch - 63ms/step
Epoch 6/40
2/2 - 0s - loss: 1.1463 - accuracy: 0.3750 - 121ms/epoch - 61ms/step
Epoch 7/40
2/2 - 0s - loss: 1.3060 - accuracy: 0.3125 - 119ms/epoch - 59ms/step
Epoch 8/40
2/2 - 0s - loss: 1.2959 - accuracy: 0.3750 - 165ms/epoch - 83ms/step
Epoch 9/40
2/2 - 0s - loss: 1.3184 - accuracy: 0.4375 - 122ms/epoch - 61ms/step
Epoch 10/40
2/2 - 0s - loss: 1.0159 - accuracy: 0.5625 - 128ms/epoch - 64ms/step
Epoch 11/40
2/2 - 0s - loss: 1.1166 - accuracy: 0.6250 - 122ms/epoch - 61ms/step
Epoch 12/40
2/2 - 0s - loss: 1.2043 - accuracy: 0.5000 - 132ms/epoch - 66ms/step
Epoch 13/40
2/2 - 0s - l

In [64]:
print("---Train Accuracy---")
model.evaluate(train_dataset,train_dataset_label)
print("---Test Accuracy---")
model.evaluate(test_dataset,test_dataset_label)

---Train Accuracy---
1/1 [==============================] - 6s 6s/step - loss: 1.3937 - accuracy: 0.2500
---Test Accuracy---
1/1 [==============================] - 0s 82ms/step - loss: 1.3937 - accuracy: 0.2500


[1.3936808109283447, 0.25]